# CIA World Factbook
En este projecto se trabajo con datos de CIA World Factbook los cuales pueden descargarse del siguiente [link](#https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db), el cual compila estadísticas de todos los países en el mundo. El Factbook contiene información demográfica de:
* population - La población al 2015.
* population_growth - El índice anual de crecimiento poblacional como porcentaje.
* area - El total de tierra y agua.

In [ ]:
%%capture #Se conecta y carga la base de datos
%load_ext sql
%sql sqlite:///factbook.db

### Muestra de tabla

In [ ]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Esta es la descripción de algunas de las columnas:

* name - Nombre del país.
* area - El total de tierra y mar del país.
* population - La población del país.
* population_growth- El crecimiento poblacional como porcentaje.
* birth_rate - El índice de naminientos del país, o número de nacimientos al año por cada 1000 personas.
* death_rate - El índice de defunciones del país, o número de defunciones al año por cada 1000 personas.
* area- El área total del país (ambos tierra y mar).
* area_land - El área de tierra del país en kilometros cuadrados.
* area_water - El área maritima del país en kilometros cuadrados.

Se calcularon las estadísticas básicas en busqueda de algún outlier en la información de los países.

In [ ]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

Se detectó que hay un país cuya población es de 0 y que hay un país con una población mayor de (700 millones), para esto se procedió a identificar los países a quienes les pertenece esta información.

In [ ]:
%%sql
SELECT name País, population Población
  FROM facts
 WHERE population==(SELECT MIN(population)
                      FROM facts)
    OR population==(SELECT MAX(population)
                      FROM facts);

Se halló que la información de la población de más de 700 millones del registro del mundo para lo cual se descartará en los siguientes análisis y con respecto a la población de 0 habitantes se puede revisar la información referente sobre el país en el siguiente [Link](#https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html)

Se calcularon las estadísticas realizadas anteriormente, esta vez excluyendo la información perteneciente a la celda *World*.

In [ ]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name <> 'World';

Se realizó el cálculo de los promedios de *población* y *área*

In [ ]:
%%sql
SELECT ROUND(AVG(population)) Promedio_Poblacion, ROUND(AVG(area),2) Promedio_area
  FROM facts
 WHERE name <> 'World';

## Población

Se desea conocer los países con una mayor densidad de población en promedio. Por lo que se idenficaran los países que se encuentran:
* Por arriba del promedio de población.
* Debajo del promedio de área.

In [ ]:
%%sql
SELECT name País, population Población, area "Territorio $km^2$"
  FROM facts
 WHERE population>(SELECT AVG(population)
                      FROM facts
                    WHERE name<>'Wolrd')
    AND "Territorio $km^2$"< (SELECT AVG(area)
                      FROM facts
                WHERE name<> 'World') 

Se halló que los paises con mayor cantidad de población sobre área provienen del continente Europeo en el caso de Germany y United Kingdom, para el caso de la zona Asiática son países que tienen cercanía unos con otros en el caso de Thailand, Bangladesh, Vietnam y Philippines a pesar de tratarse de una isla como en el caso de Japon.

Para conocer sobre de la densidad de población es inherente conocer los países con:
* Mayor población 
* Mayor índice de crecimiento.

In [ ]:
%%sql
SELECT name País_mayor_población, population Población
  FROM facts
 WHERE population== (SELECT MAX(population)
                     FROM facts
                     WHERE name <> 'World');

In [ ]:
%%sql
SELECT name País_mayor_crecimiento_poblacional, population_growth Indice_crecimiento_pob
  FROM facts
 WHERE population_growth== (SELECT MAX(population_growth)
                     FROM facts
                     WHERE name <> 'World');

Observando que los países con mayor población y un mayor índice de crecimiento en el caso de China y South Sudan respectivamente, no se encuentran dentro del listado de los paises con mayor densidad poblacional, por lo que en estos países la distribución del territorio es mayor para la población que tienen.

## Territorio
Tomando en cuenta las características del territorio surgen las siguientes preguntas:<br>
* Cuales países tienen mayor porción de agua y de tierra?
* Que países tiene superficies con mayor proporción de agua que tierra?

In [ ]:
%%sql
SELECT name País, area_water Superficie_marina
  FROM facts
 WHERE area_water > (SELECT AVG(area_water)
                     FROM facts
                     WHERE name <> 'World')
ORDER BY area_water DESC
LIMIT 5;

In [ ]:
%%sql
SELECT name País, area_land Superficie_terrestre
  FROM facts
 WHERE area_land> (SELECT AVG(area_land)
                     FROM facts
                     WHERE name <> 'World')
ORDER BY area_land DESC
LIMIT 5;

Se observó que los 5 países con mayor área de agua y de tierra se encuentran en ambas listas en los casos de Rusia, China, United States y Canada, con excepción de los países de India y Brazil siendo que la India cuenta con un mayor área de territorio que Brasil y este mismo con una mayor área de agua que la India.  

In [ ]:
%%sql
SELECT name País, area_land Superficie_terrestre, area_water Superficie_marina
  FROM facts
 WHERE area_water> area_land
 AND  name <> 'World';

Aquellos países en que el área de agua es mayor al área de tierra son el caso de las Virgin Islands y de British Indian Ocean Territory el cual es un archipielago de acuerdo a la información proporcionada en el siguiente [Link](#https://www.cia.gov/library/publications/the-world-factbook/geos/io.html)

## Demografía
Se desea conocer las estadísticas sobre la población de los países y para esto se desea conocer: <br>
* Que países tendrán más personas en su población el siguiente año?
* Cuáles países tienen un índice de muerte mayor al índice de nacimiento?
* Que países tienen el mayor índice de población/área?

In [ ]:
%%sql
SELECT COUNT(*) Paises_mayor_crecimiento_poblacional
  FROM facts
 WHERE population*population_growth> (SELECT AVG(population*population_growth)
                    FROM facts
                    WHERE name <>'World') 
 AND  name <> 'World';

In [ ]:
%%sql
SELECT name País, population Población, population_growth Indice_crecimiento, ROUND(population*population_growth) Crecimiento_poblacional
  FROM facts
 WHERE Crecimiento_poblacional> (SELECT AVG(population*population_growth)
                    FROM facts
                    WHERE name <>'World') 
 AND  name <> 'World'
 ORDER BY Crecimiento_poblacional DESC
 LIMIT 5;

Los cinco países que trandrán un mayor crecimiento poblacional provienen de Asia son el caso de India, China, Pakistan y provinientes de Africa son Nigeria y Ethiopia.

In [ ]:
%%sql
SELECT COUNT(*) 'Paises mayor mortalidad que natalidad' 
  FROM facts
 WHERE death_rate> birth_rate
 AND  name <> 'World';

In [ ]:
%%sql
SELECT name País, death_rate, birth_rate 
  FROM facts
 WHERE death_rate> birth_rate
 AND  name <> 'World'
ORDER BY death_rate DESC;

Se observó que la mayoría de los países en que el índice de mortalidad es mayor al índice de nacimientos son países Europeos con excepción de Japón proviniente del continente asiático.

In [ ]:
%%sql
SELECT COUNT(*) Países_mayor_índice_densidad
  FROM facts
 WHERE (population/area)> (SELECT AVG(population/area)
                    FROM facts
                    WHERE name <>'World') 
 AND  name <> 'World';

In [ ]:
%%sql
SELECT name País, population/area AS "Indice densidad Hab/Km"
  FROM facts
 WHERE "Indice densidad Hab/Km"> (SELECT AVG(population/area)
                    FROM facts
                    WHERE name <>'World') 
 AND  name <> 'World'
 ORDER BY "Indice densidad Hab/Km" DESC
 LIMIT 5;

Se obtuvo como resultado que la mayoría de los 5 países con mayor índice de densidad provienen del continente asiático(Macau, Singapore, Hong Kong, Gaza Strip) con exepción de Monaco que pertenece al continenete europeo. 

## Conclusión

Con los diversos análisis obtenidos se muestra un crecimiento mayor crecimiento poblacional en los países del continente asiático junto con índices de densidad superiores al promedio sino de los más altos en el mundo a pesar de ser aquellos que cuentan con una extensión terriotorial mayor al promedio que a otros países. 